#### FinanceDataReader
- 주가 데이터를 크롤링하는 패키지 입니다.
- pip install finance-datareader
- https://financedata.github.io/posts/finance-data-reader-users-guide.html

In [1]:
import FinanceDataReader as fdr

In [2]:
# 날짜 데이터 설정 : 2016 ~ 2020
start_year, end_year = "2016", "2021"

#### Quiz 1. 
- usd_df와 kospi_df의 공통된 날짜 데이터를 필터링해서 종가(컬럼명:"Close") 기준으로 상관계수를 구하세요.
- usd_df과 kospi_df의 날짜 데이터를 공통된 날짜 데이터로 필터링해서 사용하세요.
- 소수 3째자리까지 반올림해서 출력

- 데이터 크롤링

In [3]:
# 원/달러 환율
usd_df = fdr.DataReader("USD/KRW", start_year, end_year)

In [4]:
# 코스피
kospi_df = fdr.DataReader("KS11", start_year, end_year)

In [5]:
# 날짜가 같은 데이터만 출력되도록 필터링
filtered_usd_df = usd_df.loc[usd_df.index & kospi_df.index]

In [6]:
# 상관계수 구하기
np.round(np.corrcoef(filtered_usd_df["Close"], kospi_df["Close"])[0, 1], 3)

-0.699

#### Quiz 2.
- 2016년부터 2020년까지 10000달러를 애플 주식 50%, TLT 50%의 비율로 투자했을 경우 투자 수익금을 출력하세요.
- 한달에 한번씩 마지막 날에 리밸런싱합니다.

- 데이터 수집

In [7]:
# 애플 주가
apple_df = fdr.DataReader("AAPL", str(start_year), str(end_year))

In [8]:
# TLT 미국 20년 국채
tlt_df = fdr.DataReader("TLT", start_year, end_year)

In [9]:
len(apple_df), len(tlt_df)

(1260, 1259)

- apple_df와 tlt_df의 데이터의 갯수가 다릅니다.
- 날짜를 기준으로 두개의 데이터 프레임의 교집합 데이터를 아래와 같이 출력

In [10]:
# 같은 날짜 데이터로 필터링
apple_df = apple_df.loc[apple_df.index & tlt_df.index]
tlt_df = tlt_df.loc[apple_df.index & tlt_df.index]
len(apple_df), len(tlt_df)

(1259, 1259)

In [11]:
# 날짜, 종가 데이터 프레임 생성
result_df = pd.concat([apple_df.reset_index()[["Close"]].rename(columns={"Close":"apple_price"}), 
                       tlt_df.reset_index()[["Close"]].rename(columns={"Close":"tlt_price"})], axis=1)
result_df.index = apple_df.index
result_df.tail()

,apple_price,tlt_price
Date,,
2020-12-24,131.97,157.29
2020-12-28,136.69,157.36
2020-12-29,134.87,157.16
2020-12-30,133.72,157.50
2020-12-31,132.69,157.73


- 월별 마지막 날 데이터만 출력

In [12]:
# 월 단위 날짜 생성
date = pd.date_range(start=start_year, end=end_year, freq='M')
dates = date.strftime('%Y-%m')
dates[:5]

Index(['2016-01', '2016-02', '2016-03', '2016-04', '2016-05'], dtype='object')

In [13]:
# 마지막날 데이터 출력
last_dates = []
for date in dates:
    last_date = str(result_df.loc[date].index[-1]).split(" ")[0]
    last_dates.append(last_date)
last_dates[:5]

['2016-01-29', '2016-02-29', '2016-03-31', '2016-04-29', '2016-05-31']

In [14]:
result_last_df = result_df.loc[last_dates]
result_last_df.head()

,apple_price,tlt_price
Date,,
2016-01-29,24.34,127.30
2016-02-29,24.17,130.98
2016-03-31,27.25,130.61
2016-04-29,23.43,129.38
2016-05-31,24.96,130.16


- 10000 달러를 애플 주식과 TLT를 50:50으로 한달에 한번씩 투자했을때 월별 수익을 출력하는 데이터 프레임

In [15]:
df = result_last_df.reset_index()

In [16]:
original_asset = 10000
asset = original_asset

rate_1, rate_2 = 0.5, 0.5

assets = []
dollers = []
apple_stocks = []
tlt_stocks = []

for idx, data in df.iterrows():
    
    if idx > 0:
        asset = apple_stock * data["apple_price"] + tlt_stock * data["tlt_price"] + doller
    
    apple_stock = (asset * rate_1) // data["apple_price"]
    apple_stocks.append(apple_stock)
    
    tlt_stock = (asset * rate_2) // data["tlt_price"]
    tlt_stocks.append(tlt_stock)
    
    doller = np.round((asset * rate_1) % data["apple_price"] + (asset * rate_2) % data["tlt_price"], 2)
    dollers.append(doller)
    assets.append(asset)

In [17]:
df["apple_stock"] = apple_stocks
df["tlt_stock"] = tlt_stocks
df["doller"] = dollers
df["asset"] = assets
df

,Date,apple_price,tlt_price,apple_stock,tlt_stock,doller,asset
0,2016-01-29,24.34,127.30,205.0,39.0,45.60,10000.00
1,2016-02-29,24.17,130.98,209.0,38.0,79.90,10108.67
2,2016-03-31,27.25,130.61,197.0,41.0,15.07,10738.33
3,2016-04-29,23.43,129.38,212.0,38.0,51.76,9935.36
4,2016-05-31,24.96,130.16,206.0,39.0,71.36,10289.36
5,2016-06-30,23.90,138.90,217.0,37.0,86.26,10411.86
6,2016-07-29,26.05,141.56,210.0,38.0,127.05,10976.83
7,2016-08-31,26.52,139.87,207.0,39.0,66.74,11011.31
8,2016-09-30,28.26,137.51,199.0,41.0,17.80,11279.45
9,2016-10-31,28.39,131.25,194.0,42.0,28.50,11048.66


In [19]:
# 수익률 출력
np.round(asset / original_asset * 100, 2)

(278.6, '%')